ARBOL DE DECISION CON GRIDSEARCH


In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("../data/datos_modificados.csv")
data=data.drop(columns="id")

In [3]:
data_usable=data[data["x_e_out [-]"].notnull()].copy()
prediction=data[data["x_e_out [-]"].isnull()].copy()

Arbol de decision

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler

In [5]:
# Divido los datos en características (X) y columna objetivo (y)
X = data_usable.drop('x_e_out [-]', axis=1)
y = data_usable['x_e_out [-]']

# Identificar las columnas categóricas
categorical_cols = ['author', 'geometry']

# Definir el transformador para la codificación one-hot
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')

# Aplicar la codificación one-hot a las variables categóricas
X_encoded = ct.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, shuffle=True)


In [7]:
from sklearn.model_selection import GridSearchCV

# Definir los valores a probar para cada hiperparámetro
param_grid = {
    'max_depth': [3, 5, 7, 9,15],  # Valores posibles para la profundidad máxima del árbol
    'min_samples_split': [2, 5, 10,15,20],  # Valores posibles para el número mínimo de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2, 4,6,8]  # Valores posibles para el número mínimo de muestras requeridas en cada hoja
}

# Crear una instancia del modelo
model = DecisionTreeRegressor(random_state=42)

# Realizar la búsqueda de cuadrícula con validación cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Obtener el mejor modelo ajustado con los mejores hiperparámetros
best_model = grid_search.best_estimator_

# Evaluar el modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE con mejores hiperparámetros:", rmse)


Mejores hiperparámetros: {'max_depth': 9, 'min_samples_leaf': 6, 'min_samples_split': 20}
RMSE con mejores hiperparámetros: 0.07850422278685386
